In [52]:
import yaml
import requests
import time
from datetime import datetime, timedelta, date
import pandas as pd

Les données sont mises à disposition sur https://github.com/opencovid19-fr/data par sante-publique-france

## Automatisation de la récupération des données

In [53]:
r = requests.get('https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/2020-01-24.yaml')
r.status_code

200

In [54]:
r.text

'date: "2020-01-24"\nsource:\n  nom: Santé publique France\n  url: https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine\n  archive: https://web.archive.org/web/20200124202351/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine\ndonneesNationales: # Voir archive 2 en bas de page (après le bulletin du 12/02)\n  casConfirmes: 3 # cas mise à jour suite à la publication du 27/01 de la SPF qui mentionnent ces 3 cas comme existant depuis le 24/01\n  hospitalises: 3\n  deces: 0\ndonneesRegionales: # Voir archive 2 en bas de page (après le bulletin du 12/02)\n  - nom: Île-de-France\n    code: REG-11\n    casConfirmes: 2\n    hospitalises: 2\n  - nom: Nouvelle-Aquitaine\n    code: REG-7

In [55]:
y = yaml.load(r.text, Loader=yaml.FullLoader)

In [56]:
y

{'date': '2020-01-24',
 'source': {'nom': 'Santé publique France',
  'url': 'https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine',
  'archive': 'https://web.archive.org/web/20200124202351/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/cas-de-pneumonies-associees-a-un-nouveau-coronavirus-2019-ncov-a-wuhan-en-chine'},
 'donneesNationales': {'casConfirmes': 3, 'hospitalises': 3, 'deces': 0},
 'donneesRegionales': [{'nom': 'Île-de-France',
   'code': 'REG-11',
   'casConfirmes': 2,
   'hospitalises': 2},
  {'nom': 'Nouvelle-Aquitaine',
   'code': 'REG-75',
   'casConfirmes': 1,
   'hospitalises': 1,
   'donneesRegionales': None}],
 'donneesDepartementales': [{'nom': 'Gironde',
   'code': 'DEP-33',
   'casConfirmes': 1,
   'hospitalises': 1}],
 'donneesM

### Avec une seule fonction ...

In [57]:
r = requests.get('https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/2020-03-24.yaml')
if (r.status_code==200):
    print (yaml.load(r.text, Loader=yaml.FullLoader))

{'date': datetime.date(2020, 3, 24), 'source': {'nom': 'Santé publique France', 'url': 'https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/infection-au-nouveau-coronavirus-sars-cov-2-covid-19-france-et-monde', 'archive': 'https://web.archive.org/web/20200324193012/https://www.santepubliquefrance.fr/maladies-et-traumatismes/maladies-et-infections-respiratoires/infection-a-coronavirus/articles/infection-au-nouveau-coronavirus-sars-cov-2-covid-19-france-et-monde'}, 'donneesNationales': {'casConfirmes': 22302, 'deces': 1100}, 'donneesRegionales': [{'nom': 'Auvergne-Rhône-Alpes', 'code': 'REG-84', 'casConfirmes': 1857}, {'nom': 'Bourgogne-Franche-Comté', 'code': 'REG-27', 'casConfirmes': 1510}, {'nom': 'Bretagne', 'code': 'REG-53', 'casConfirmes': 526}, {'nom': 'Centre-Val de Loire', 'code': 'REG-24', 'casConfirmes': 450}, {'nom': 'Corse', 'code': 'REG-94', 'casConfirmes': 217}, {'nom': 'Grand Est', 'code': 'REG-

In [59]:
hier = date.today() + timedelta(-1)
shier = hier.strftime("%Y-%m-%d")
fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/' + shier + '.yaml'

## Création d'une fonction qui récupère les données d'hier sur le Covid19

In [60]:
def RecupCovid19Hier():
    hier = date.today() + timedelta(-1)
    shier = hier.strftime("%Y-%m-%d")
    fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/' + shier + '.yaml'
    out = ""
    r = requests.get(fichier)
    if (r.status_code==200):
       out = yaml.load(r.text, Loader=yaml.FullLoader)
    return out

In [61]:
jsonout = RecupCovid19Hier()
jsonout

''

In [62]:
jsonout['date'].strftime("%Y-%m-%d")

TypeError: string indices must be integers

In [34]:
jsonout['donneesNationales']['casConfirmes']

22302

## Sérialisation des données dans un dataframe

In [61]:
A = pd.DataFrame()
A = A.append({
                'Date' : jsonout['date'].strftime("%Y-%m-%d") , 
                'FR Cas Confirmés' : jsonout['donneesNationales']['casConfirmes'] , 
                'FR Décès' : jsonout['donneesNationales']['deces'], 
                'WW Cas Confirmés' : jsonout['donneesMondiales']['casConfirmes'],
                'WW Décès' : jsonout['donneesMondiales']['deces']
             } , 
             ignore_index=True)
A

,Date,FR Cas Confirmés,FR Décès,WW Cas Confirmés,WW Décès
0,2020-03-24,22302.0,1100.0,378041.0,16365.0


In [75]:
datetime_object = datetime.strptime(StartingDate, '%Y-%m-%d')
datetime_object

datetime.datetime(2020, 1, 24, 0, 0)

In [79]:
date_str = "2020-01-24"
date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
print(date_object)

2020-01-24


## Fonction globale / récupération des données brutes de Santé Publique France

In [124]:
# INITIALISATION DE VARIABLES GLOBALES
sDateDebut = "2020-01-23"
sAujourdhui = date.today().strftime("%Y-%m-%d")
dDemarrageDate = datetime.fromisoformat(sDateDebut) 
cols = ['Date', 
        'FR Tot Cas Confirmés', 
        'FR Tot Décès', 
        'WW Tot Cas Confirmés', 
        'WW Tot Décès', 
        'Source',
        'FR Tot Guéris',
        'FR Tot Hospitalisés'
       ]

# FONCTIONS
def recupDataWithException2(_jsonout, _index1, _index2):
    try:
        data = _jsonout[_index1][_index2]
    except:
        data = 0
    return data

def AjouteSantePubliqueFrLigne(jsonout, df):
    df = df.append({
                    cols[0] : jsonout['date'] , 
                    cols[1] : recupDataWithException2(jsonout, 'donneesNationales', 'casConfirmes') , 
                    cols[2] : recupDataWithException2(jsonout, 'donneesNationales', 'deces'), 
                    cols[3] : recupDataWithException2(jsonout,'donneesMondiales', 'casConfirmes'),
                    cols[4] : recupDataWithException2(jsonout,'donneesMondiales', 'deces'),
                    cols[5] : 'sante-publique-france',
                    cols[6] : 0,
                    cols[7] : 0
                 } , 
                 ignore_index=True)
    return df

def ajouteLigneVide(df, madate):
    df = df.append({
                    cols[0] : madate , 
                    cols[1] : 0,
                    cols[2] : 0, 
                    cols[3] : 0,
                    cols[4] : 0,
                    cols[5] : 'no-data',
                    cols[6] : 0,
                    cols[7] : 0
                 } , 
                 ignore_index=True)
    return df

# RECUPERATION DONNEES SANTE PUBLIQUE FRANCE
def RecupereDonneesSantePubliqueFrance():
    print ('--> Démarrage du process')
    sDateParcours = sDateDebut
    i=1
    requesReturnCode=200
    yamls = pd.DataFrame()
    while (sDateParcours != sAujourdhui):
        myDate = dDemarrageDate + timedelta(days=i)
        sDateParcours = myDate.strftime("%Y-%m-%d")
        fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/sante-publique-france/' + sDateParcours + '.yaml'
        #print ('--> sante-publique-france / ', sDateParcours)
        req = requests.get(fichier)
        requesReturnCode = req.status_code
        if (requesReturnCode==200):
            yamlout = yaml.load(req.text, Loader=yaml.FullLoader)
            yamls = AjouteSantePubliqueFrLigne(yamlout, yamls)
        else:
            print ("(*) Pas de données pour ", myDate.strftime("%Y-%m-%d"))
            # ajoute une ligne vide
            yamls = ajouteLigneVide(yamls, myDate.strftime("%Y-%m-%d"))
        i=i+1
    print ('--> Fin du process')
    return yamls
    
data = RecupereDonneesSantePubliqueFrance()
data

--> Démarrage du process
(*) Pas de données pour  2020-02-01
(*) Pas de données pour  2020-02-02
(*) Pas de données pour  2020-02-04
(*) Pas de données pour  2020-02-09
(*) Pas de données pour  2020-02-13
(*) Pas de données pour  2020-02-14
(*) Pas de données pour  2020-02-15
(*) Pas de données pour  2020-02-16
(*) Pas de données pour  2020-02-17
(*) Pas de données pour  2020-02-18
(*) Pas de données pour  2020-02-19
(*) Pas de données pour  2020-02-21
(*) Pas de données pour  2020-02-22
(*) Pas de données pour  2020-02-23
(*) Pas de données pour  2020-02-24
(*) Pas de données pour  2020-02-27
(*) Pas de données pour  2020-03-01
(*) Pas de données pour  2020-03-29
--> Fin du process


,Date,FR Tot Cas Confirmés,FR Tot Décès,FR Tot Guéris,FR Tot Hospitalisés,Source,WW Tot Cas Confirmés,WW Tot Décès
0,2020-01-24,3.0,0.0,0.0,0.0,sante-publique-france,897.0,26.0
1,2020-01-25,3.0,0.0,0.0,0.0,sante-publique-france,1329.0,41.0
2,2020-01-26,3.0,0.0,0.0,0.0,sante-publique-france,2026.0,56.0
3,2020-01-27,3.0,0.0,0.0,0.0,sante-publique-france,2820.0,81.0
4,2020-01-28,4.0,0.0,0.0,0.0,sante-publique-france,0.0,0.0
...,...,...,...,...,...,...,...,...
61,2020-03-25,25233.0,1331.0,0.0,0.0,sante-publique-france,416916.0,18865.0
62,2020-03-26,29155.0,1696.0,0.0,0.0,sante-publique-france,467710.0,20947.0
63,2020-03-27,32964.0,1995.0,0.0,0.0,sante-publique-france,528025.0,23672.0
64,2020-03-28,37575.0,2314.0,0.0,0.0,sante-publique-france,591971.0,27090.0


## Consolidation avec les données du ministère de la santé

On va récupérer les données du ministère et les consolider avec celles de Santé publique France

https://raw.githubusercontent.com/opencovid19-fr/data/master/ministere-sante/2020-01-24.yaml

In [126]:
# RECUPERATION DONNEES SANTE PUBLIQUE FRANCE
def RecupereDonneesMinistereSante(yamls):
    i=1
    dateParcours = dateDebut
    print ('--> Démarrage du process')
    while (dateParcours != aujourdhui):
        myDate = dStartingDate + timedelta(days=i)
        dateParcours = myDate.strftime("%Y-%m-%d")
        #print("Traintement date: ",  dateParcours)
        fichier = 'https://raw.githubusercontent.com/opencovid19-fr/data/master/ministere-sante/' + dateParcours + '.yaml'
        req = requests.get(fichier)
        requesReturnCode = req.status_code
        if (requesReturnCode==200):
            yamlout = yaml.load(req.text, Loader=yaml.FullLoader)
            if (yamls[cols[5]][i-1] == "no-data"):
                print ("(*) Modifie les données manquantes pour la date du ", myDate.strftime("%Y-%m-%d"))
                # Modifie toute la ligne
                yamls[cols[1]][i-1] = recupDataWithException2(yamlout, 'donneesNationales', 'casConfirmes')
                yamls[cols[2]][i-1] = recupDataWithException2(yamlout, 'donneesNationales', 'deces')
                yamls[cols[3]][i-1] = recupDataWithException2(yamlout, 'donneesMondiales', 'casConfirmes')
                yamls[cols[4]][i-1] = recupDataWithException2(yamlout, 'donneesMondiales', 'deces')
                yamls[cols[5]][i-1] = "ministere-sante"

            # Ajoute les nouveaux champs
            yamls[cols[6]][i-1] = recupDataWithException2(yamlout, 'donneesNationales', 'hospitalises')
            yamls[cols[7]][i-1] = recupDataWithException2(yamlout, 'donneesNationales', 'gueris')
        i=i+1
    print ('--> Fin du process')
    return yamls
    
data = RecupereDonneesMinistereSante(data)

--> Démarrage du process


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(*) Modifie les données manquantes pour la date du  2020-02-02


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value i

(*) Modifie les données manquantes pour la date du  2020-02-04
(*) Modifie les données manquantes pour la date du  2020-02-09
(*) Modifie les données manquantes pour la date du  2020-02-13
(*) Modifie les données manquantes pour la date du  2020-02-15
(*) Modifie les données manquantes pour la date du  2020-02-17
(*) Modifie les données manquantes pour la date du  2020-02-19
(*) Modifie les données manquantes pour la date du  2020-02-21
(*) Modifie les données manquantes pour la date du  2020-02-23
(*) Modifie les données manquantes pour la date du  2020-02-24
(*) Modifie les données manquantes pour la date du  2020-02-27
(*) Modifie les données manquantes pour la date du  2020-03-01
--> Fin du process


,Date,FR Tot Cas Confirmés,FR Tot Décès,FR Tot Guéris,FR Tot Hospitalisés,Source,WW Tot Cas Confirmés,WW Tot Décès
0,2020-01-24,3.0,0.0,3.0,0.0,sante-publique-france,897.0,26.0
1,2020-01-25,3.0,0.0,0.0,0.0,sante-publique-france,1329.0,41.0
2,2020-01-26,3.0,0.0,0.0,0.0,sante-publique-france,2026.0,56.0
3,2020-01-27,3.0,0.0,3.0,0.0,sante-publique-france,2820.0,81.0
4,2020-01-28,4.0,0.0,4.0,0.0,sante-publique-france,0.0,0.0
...,...,...,...,...,...,...,...,...
61,2020-03-25,25233.0,1331.0,11539.0,3900.0,sante-publique-france,416916.0,18865.0
62,2020-03-26,29155.0,1696.0,13904.0,4948.0,sante-publique-france,467710.0,20947.0
63,2020-03-27,32964.0,1995.0,15732.0,0.0,sante-publique-france,528025.0,23672.0
64,2020-03-28,37575.0,2314.0,17620.0,0.0,sante-publique-france,591971.0,27090.0


## Comble les "trous" sur les no-data

In [142]:
def Bouchetrous(yamls):
    # On reparcourre toutes les lignes déjà récupérées précédemment
    i=1
    for index, row in yamls.iterrows():
        if (yamls[cols[5]][i-1] == "no-data"):
            for j in [1, 2, 3, 4, 6, 7]:
                yamls[cols[j]][i-1] = yamls[cols[j]][i-2] 
        i += 1
    return yamls

data = Bouchetrous(data)
data.to_csv("datafr.csv", index=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [130]:
yamls

,Date,FR Tot Cas Confirmés,FR Tot Décès,FR Tot Guéris,FR Tot Hospitalisés,Source,WW Tot Cas Confirmés,WW Tot Décès
0,2020-01-24,3.0,0.0,0.0,0.0,sante-publique-france,897.0,26.0
1,2020-01-25,3.0,0.0,0.0,0.0,sante-publique-france,1329.0,41.0
2,2020-01-26,3.0,0.0,0.0,0.0,sante-publique-france,2026.0,56.0
3,2020-01-27,3.0,0.0,0.0,0.0,sante-publique-france,2820.0,81.0
4,2020-01-28,4.0,0.0,0.0,0.0,sante-publique-france,0.0,0.0
...,...,...,...,...,...,...,...,...
61,2020-03-25,25233.0,1331.0,0.0,0.0,sante-publique-france,416916.0,18865.0
62,2020-03-26,29155.0,1696.0,0.0,0.0,sante-publique-france,467710.0,20947.0
63,2020-03-27,32964.0,1995.0,0.0,0.0,sante-publique-france,528025.0,23672.0
64,2020-03-28,37575.0,2314.0,0.0,0.0,sante-publique-france,591971.0,27090.0


In [138]:
#yamls[cols[j]][1]
yamls.loc[1][cols[1]]

3.0